Lectura de xml

In [1]:
import pandas as pd
import xml.etree.ElementTree as etree


tree = etree.parse("pu-cs-fal07.xml")
#tree = etree.parse("lums-sum17.xml")
root = tree.getroot()

columns_problem = ["Name", "NrDays", "SlotsPerDay", "NrWeeks"]
columns_optimization = ["Time", "Room", "Distribution", "Student"]
columns_rooms = ["RoomId", "Capacity", "PreviousRoomIdList", "TravelTimeValueList", "UnavailableDaysList", "UnavailableStartList", "UnavailableLengthList", "UnavailableWeeksList"]
columns_courses = ["CourseId", "ConfigIdList"]
columns_configs = ["ConfigId", "SubpartIdList"]
columns_subparts = ["SubpartId", "ClassIdList"]
columns_classes = ["ClassId", "ClassLimit", "ClassParent"]
columns_distributions = ["DistributionId", "DistributionType", "DistributionRequired", "DistributionPenalty", "ClassesList"]
columns_students = ["StudentId", "CoursesList"]
columns_relation_class_room = ["ClassId", "RoomId", "RoomPenalty"]
columns_relation_class_time = ["ClassId", "TimeDays", "TimeStart", "TimeLength", "TimeWeeks", "TimePenalty"]

df_problem = pd.DataFrame(columns = columns_problem)
df_optimization = pd.DataFrame(columns = columns_optimization)
df_rooms = pd.DataFrame(columns = columns_rooms)
df_courses = pd.DataFrame(columns = columns_courses)
df_configs = pd.DataFrame(columns = columns_configs)
df_subparts = pd.DataFrame(columns = columns_subparts)
df_classes = pd.DataFrame(columns = columns_classes)
df_distributions = pd.DataFrame(columns = columns_distributions)
df_students = pd.DataFrame(columns = columns_students)
df_relation_class_room = pd.DataFrame(columns = columns_relation_class_room)
df_relation_class_time = pd.DataFrame(columns = columns_relation_class_time)




df_problem = df_problem.append({"Name": root.attrib.get("name"), "NrDays": root.attrib.get("nrDays"), "SlotsPerDay": root.attrib.get("slotsPerDay"), "NrWeeks": root.attrib.get("nrWeeks")}, ignore_index = True)

for node in root:
    if node.tag == "optimization":
        df_optimization = df_optimization.append({"Time": node.attrib.get("time"), "Room": node.attrib.get("room"), "Distribution": node.attrib.get("distribution"), "Student": node.attrib.get("student")}, ignore_index = True)
    
    elif node.tag == "rooms":
        for room in node:
            roomid = room.attrib.get("id")
            cap = room.attrib.get("capacity")
            prevroomlist = []
            valuelist = []
            dayslist = []
            startlist = []
            lengthlist = []
            weekslist = []
            if len(list(child.tag for child in room.iter() if child is not room)) > 0:
                for child in room:
                    if child.tag == "travel":
                        prevroomid = child.attrib.get("room")
                        value = child.attrib.get("value")
                        prevroomlist.append(prevroomid)
                        valuelist.append(value)
                    elif child.tag == "unavailable":
                        days = child.attrib.get("days")
                        start = child.attrib.get("start")
                        length = child.attrib.get("length")
                        weeks = child.attrib.get("weeks")
                        dayslist.append(days)
                        startlist.append(start)
                        lengthlist.append(length)
                        weekslist.append(weeks)
            if len(prevroomlist) == 0:
                prevroomlist = None
            if len(valuelist) == 0:
                valuelist = None
            if len(dayslist) == 0:
                dayslist = None
            if len(startlist) == 0:
                startlist = None
            if len(lengthlist) == 0:
                lengthlist = None
            if len(weekslist) == 0:
                weekslist = None
            df_rooms = df_rooms.append(pd.Series([roomid, cap, prevroomlist, valuelist, dayslist, startlist, lengthlist, weekslist], index = columns_rooms), ignore_index = True)
            
    elif node.tag == "courses":
        for course in node:
            #print(course.tag)
            courseid = course.attrib.get("id")
            configlist = []
            for config in course:
                #print(config.tag)
                configid = config.attrib.get("id")
                configlist.append(configid)
                subpartlist = []
                for subpart in config:
                    #print(subpart.tag)
                    subpartid = subpart.attrib.get("id")
                    subpartlist.append(subpartid)
                    claselist = []
                    for clase in subpart:
                        #print(clase.tag)
                        claseid = clase.attrib.get("id")
                        claselist.append(claseid)
                        claselimit = clase.attrib.get("limit")
                        claseparent = clase.attrib.get("parent") if clase.attrib.get("parent") is not None else None
                        df_classes = df_classes.append(pd.Series([claseid, claselimit, claseparent], index = columns_classes), ignore_index = True)
                        if len(list(child.tag for child in clase.iter() if child is not clase)) > 0:
                            for child in clase:
                                #print(child.tag)
                                if child.tag == "room":
                                    roomid = child.attrib.get("id")
                                    roompenalty = child.attrib.get("penalty")
                                    df_relation_class_room = df_relation_class_room.append(pd.Series([claseid, roomid, roompenalty], index = columns_relation_class_room), ignore_index = True)
                                elif child.tag == "time":
                                    timedays = child.attrib.get("days")
                                    timestart = child.attrib.get("start")
                                    timelength = child.attrib.get("length")
                                    timeweeks = child.attrib.get("weeks")
                                    timepenalty = child.attrib.get("penalty")
                                    df_relation_class_time = df_relation_class_time.append(pd.Series([claseid, timedays, timestart, timelength, timeweeks, timepenalty], index = columns_relation_class_time), ignore_index = True)
                    df_subparts = df_subparts.append(pd.Series([subpartid, claselist], index = columns_subparts), ignore_index = True)
                df_configs = df_configs.append(pd.Series([configid, subpartlist], index = columns_configs), ignore_index = True)
            df_courses = df_courses.append(pd.Series([courseid, configlist], index = columns_courses), ignore_index = True)
                                    
    elif node.tag == "distributions":
        contador = 1
        for distribution in node:
            #print(distribution.tag)
            distribid = contador
            distribtype = distribution.attrib.get("type")
            distribrequired = distribution.attrib.get("required") if distribution.attrib.get("required") is not None else None
            distribpenalty = distribution.attrib.get("penalty") if distribution.attrib.get("penalty") is not None else None
            listaclases = []
            for clase in distribution:
                #print(clase.tag)
                listaclases.append(clase.attrib.get("id"))
            df_distributions = df_distributions.append(pd.Series([distribid, distribtype, distribrequired, distribpenalty, listaclases], index = columns_distributions), ignore_index = True)
            contador += 1
             
    elif node.tag == "students":
        for student in node:
            studentid = student.attrib.get("id")
            listacourses = []
            for course in student:
                listacourses.append(course.attrib.get("id"))
            df_students = df_students.append(pd.Series([studentid, listacourses], index = columns_students), ignore_index = True)
            
            


print("################################################################################################################")
print("DataFrame Problem")
display(df_problem)
print("################################################################################################################")
print("DataFrame Optimization")
display(df_optimization)
print("################################################################################################################")
print("DataFrame Rooms")
display(df_rooms)
print("################################################################################################################")
print("DataFrame Courses")
display(df_courses)
print("################################################################################################################")
print("DataFrame Configs")
display(df_configs)
print("################################################################################################################")
print("DataFrame Subparts")
display(df_subparts)
print("################################################################################################################")
print("DataFrame Classes")
display(df_classes)
print("################################################################################################################")
print("DataFrame Distributions")
display(df_distributions)
print("################################################################################################################")
print("DataFrame Students")
display(df_students)
print("################################################################################################################")
print("DataFrame RelationClassRoom")
display(df_relation_class_room)
print("################################################################################################################")
print("DataFrame RelationClassTime")
display(df_relation_class_time)
print("################################################################################################################")

################################################################################################################
DataFrame Problem


,Name,NrDays,SlotsPerDay,NrWeeks
0,pu-cs-fal07,7,288,15


################################################################################################################
DataFrame Optimization


,Time,Room,Distribution,Student
0,1,1,10,10


################################################################################################################
DataFrame Rooms


...

################################################################################################################
DataFrame RelationClassRoom


,ClassId,RoomId,RoomPenalty
0,1,10,0
1,1,11,0
2,1,12,0
3,2,10,0
4,2,11,0
...,...,...,...
385,173,2,0
386,173,3,0
387,174,1,44
388,174,2,0


################################################################################################################
DataFrame RelationClassTime


,ClassId,TimeDays,TimeStart,TimeLength,TimeWeeks,TimePenalty
0,1,0100000,90,22,111111111111111,0
1,1,0010000,90,22,111111111111111,0
2,1,0001000,90,22,111111111111111,0
3,1,0000100,90,22,111111111111111,0
4,1,0100000,114,22,111111111111111,0
...,...,...,...,...,...,...
2953,174,0101000,186,10,111111110000000,16
2954,174,1010000,198,10,111111110000000,16
2955,174,1000100,198,10,111111110000000,16
2956,174,0101000,198,10,111111110000000,16


################################################################################################################


Lectura xml solucion

Analisis de datos

In [2]:
## Duracion en slots de clases:

df_relation_class_time["TimeLength"] = df_relation_class_time["TimeLength"].astype(int)
print("Promedio:")
display(df_relation_class_time["TimeLength"].mean())
print("Min:")
display(df_relation_class_time["TimeLength"].min())
print("Max:")
display(df_relation_class_time["TimeLength"].max())
print("Tiempo de inicio")
display(df_relation_class_time.groupby("TimeStart").count())
x = df_relation_class_time.groupby("TimeStart").count()
print("Count")
display(df_relation_class_time["TimeLength"].value_counts())
print("Mediana:")
display(df_relation_class_time["TimeLength"].median())
print("Cuartiles:")
display(df_relation_class_time["TimeLength"].quantile([0, 0.25, 0.5, 0.75, 1]))
print("Var:")
display(df_relation_class_time["TimeLength"].var())
print("Std:")
display(df_relation_class_time["TimeLength"].std())

Promedio:


18.17816091954023

Min:


10

Max:


...

0.00    10.0
0.25    10.0
0.50    22.0
0.75    22.0
1.00    22.0
Name: TimeLength, dtype: float64

Var:


29.62093648812983

Std:


5.442511964904609

In [3]:
##SPFf=subpartes de la configuracion f
#Cantidad de subpartes de una configuración
dicc_SPFf = {}
for index, col in df_configs.iterrows():
    dicc_SPFf[col["ConfigId"]] = [int(x) for x in col["SubpartIdList"]]
print(dicc_SPFf)


{'1': [1], '2': [2, 3], '3': [4], '4': [5, 6], '5': [7, 8], '6': [9, 10], '7': [11], '8': [12, 13], '9': [14, 15], '10': [16, 17], '11': [18, 19], '12': [20, 21], '13': [22, 23], '14': [24], '15': [25, 26], '16': [27, 28], '17': [29], '18': [30, 31], '19': [32, 33], '20': [34], '21': [35, 36], '22': [37], '23': [38], '24': [39], '25': [40, 41], '26': [42, 43], '27': [44, 45], '28': [46, 47], '29': [48], '30': [49], '31': [50], '32': [51], '33': [52], '34': [53, 54], '35': [55], '36': [56], '37': [57], '38': [58], '39': [59], '40': [60], '41': [61], '42': [62], '43': [63], '44': [64]}


In [4]:
#DFe=configuraciones de cursos demandados por el estudiante e
#StudentId	CoursesList
dicc_DFe = {}
for index, col in df_students.iterrows():
    dicc_DFe[int(col["StudentId"])] = [int(x) for x in col["CoursesList"]]
print(dicc_DFe)

{1: [4], 2: [1], 3: [20, 26], 4: [5], 5: [21, 23], 6: [4], 7: [21, 23, 44], 8: [17], 9: [18], 10: [1], 11: [18, 24, 25], 12: [25], 13: [20], 14: [4, 9], 15: [1], 16: [23], 17: [14], 18: [14], 19: [18, 25], 20: [20, 24], 21: [16, 23], 22: [1], 23: [14], 24: [14], 25: [20, 23, 24], 26: [1], 27: [20, 24], 28: [1], 29: [3], 30: [3], 31: [14], 32: [21], 33: [1], 34: [1], 35: [4], 36: [1], 37: [1], 38: [3], 39: [20], 40: [1], 41: [20, 23], 42: [1], 43: [16], 44: [3], 45: [17], 46: [14], 47: [1], 48: [5, 10], 49: [3], 50: [20, 26], 51: [1], 52: [17], 53: [1], 54: [14], 55: [14], 56: [1], 57: [14], 58: [14], 59: [14], 60: [3], 61: [3], 62: [1], 63: [17], 64: [14], 65: [17], 66: [3], 67: [14], 68: [14], 69: [14], 70: [1], 71: [1], 72: [14], 73: [14], 74: [14], 75: [14], 76: [1], 77: [3], 78: [20], 79: [14], 80: [3], 81: [1], 82: [3], 83: [1], 84: [1], 85: [3], 86: [14], 87: [6], 88: [3], 89: [3], 90: [3], 91: [1], 92: [1], 93: [1], 94: [1], 95: [3], 96: [1], 97: [1], 98: [14], 99: [1], 100: [3]

In [5]:
#UTs=tiempo no disponible para la sala s
#RoomId	UnavailableDaysList	UnavailableStartList	UnavailableLengthList	UnavailableWeeksList
dicc_UTs = {}
for index, col in df_rooms.iterrows():
    if col["UnavailableDaysList"] != None:
        dicc_UTs[col["RoomId"]] = [col["UnavailableDaysList"],col["UnavailableStartList"],col["UnavailableLengthList"],col["UnavailableWeeksList"]]
print(dicc_UTs)

{'1': [['0010000'], ['222'], ['36'], ['111111111111111']], '11': [['1000000'], ['90'], ['132'], ['111111111111111']]}


In [6]:
## TDc=tiempos en los que se puede impartir la clase c
dicc_TDc = {}
for index,row in df_relation_class_time.iterrows():
    if int(row["ClassId"]) not in dicc_TDc.keys():
        dicc_TDc[int(row["ClassId"])] = [[row["TimeDays"],row["TimeStart"],row["TimeLength"],row["TimeWeeks"],row["TimePenalty"]]]
    else:
        dicc_TDc[int(row["ClassId"])].append([row["TimeDays"],row["TimeStart"],row["TimeLength"],row["TimeWeeks"],row["TimePenalty"]])

#for i in dicc_TDc:
    #print(i,dicc_TDc[i])

dicc_TDc2 = dicc_TDc
dicc_TDc = {}
contador = 0
for e,i in enumerate(dicc_TDc2):
    for j in dicc_TDc2[i]:
        if i not in dicc_TDc.keys():
            dicc_TDc[i] = []
        dicc_TDc[i].append(contador)
        contador+=1
print(dicc_TDc)


{1: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19], 2: [20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39], 3: [40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59], 4: [60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79], 5: [80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99], 6: [100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119], 7: [120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139], 8: [140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159], 9: [160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179], 10: [180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199], 11: [200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210

In [7]:
dicc_t = {}
for index,row in df_relation_class_time.iterrows():
    dicc_t[index] = [row["ClassId"],row["TimeDays"],row["TimeStart"],row["TimeLength"],row["TimeWeeks"],row["TimePenalty"]]
print(dicc_t)

{0: ['1', '0100000', '90', 22, '111111111111111', '0'], 1: ['1', '0010000', '90', 22, '111111111111111', '0'], 2: ['1', '0001000', '90', 22, '111111111111111', '0'], 3: ['1', '0000100', '90', 22, '111111111111111', '0'], 4: ['1', '0100000', '114', 22, '111111111111111', '0'], 5: ['1', '0010000', '114', 22, '111111111111111', '0'], 6: ['1', '0001000', '114', 22, '111111111111111', '0'], 7: ['1', '0000100', '114', 22, '111111111111111', '0'], 8: ['1', '0100000', '138', 22, '111111111111111', '0'], 9: ['1', '0010000', '138', 22, '111111111111111', '0'], 10: ['1', '0001000', '138', 22, '111111111111111', '0'], 11: ['1', '0000100', '138', 22, '111111111111111', '0'], 12: ['1', '0100000', '162', 22, '111111111111111', '0'], 13: ['1', '0010000', '162', 22, '111111111111111', '0'], 14: ['1', '0001000', '162', 22, '111111111111111', '0'], 15: ['1', '0000100', '162', 22, '111111111111111', '0'], 16: ['1', '0100000', '186', 22, '111111111111111', '0'], 17: ['1', '0010000', '186', 22, '11111111111

In [34]:
## SDc=salas disponibles para la clase c

dicc_SDispc = {}
for index, row in df_relation_class_room.iterrows():
    if int(row["ClassId"]) not in dicc_SDispc.keys():
        dicc_SDispc[int(row["ClassId"])]=[int(row["RoomId"])]
    else:
        dicc_SDispc[int(row["ClassId"])].append(int(row["RoomId"]))

print(dicc_SDispc)

{1: [10, 11, 12], 2: [10, 11, 12], 3: [10, 11, 12], 4: [10, 11, 12], 5: [10, 11, 12], 6: [10, 11, 12], 7: [10, 11, 12], 8: [10, 11, 12], 9: [10, 11, 12], 10: [10, 11, 12], 11: [10, 11, 12], 12: [10, 11, 12], 13: [10, 11, 12], 14: [10, 11, 12], 15: [10, 11, 12], 16: [10, 11, 12], 17: [10, 11, 12], 18: [10, 11, 12], 19: [10, 11, 12], 20: [10, 11, 12], 21: [10, 11, 12], 22: [10, 11, 12], 23: [10, 11, 12], 24: [10, 11, 12], 25: [10, 11, 12], 26: [10, 11, 12], 27: [1], 28: [13], 29: [13], 30: [13], 31: [13], 32: [13], 33: [13], 34: [13], 35: [13], 36: [13], 37: [13], 38: [13], 39: [13], 40: [13], 41: [13], 42: [13], 43: [1, 2, 3], 44: [1, 2, 3], 45: [1, 2, 3], 46: [1, 2, 3], 47: [1, 2, 3], 48: [1, 2, 3], 49: [1, 2, 3], 50: [1, 2, 3], 51: [7], 52: [7], 53: [7], 54: [7], 55: [7], 56: [7], 57: [7], 58: [7], 59: [1, 2, 3], 60: [1, 2, 3], 61: [1, 2, 3], 62: [1, 2, 3], 63: [1, 2, 3], 64: [1, 2, 3], 65: [1, 2, 3], 66: [1, 2, 3], 67: [1, 2, 3], 68: [7], 69: [7], 70: [7], 71: [7], 72: [7], 73: [7], 

In [9]:
#SAc=clases c de la distribución same attendees
dicc_SAc = {}
for index, col in df_distributions.iterrows():
    if col["DistributionType"] == "SameAttendees":
        dicc_SAc[col["ClassesList"][0]] = [int(x) for x in col["ClassesList"]] 

print(dicc_SAc)
        

{'140': [140, 155, 156], '145': [145, 165], '28': [28, 27], '29': [29, 27], '51': [51, 43], '52': [52, 44], '53': [53, 45], '54': [54, 46], '55': [55, 47], '56': [56, 48], '57': [57, 49], '58': [58, 50], '68': [68, 59], '69': [69, 60], '70': [70, 61], '71': [71, 62], '72': [72, 63], '73': [73, 64], '74': [74, 65], '75': [75, 66], '76': [76, 67], '78': [78, 77], '79': [79, 77], '82': [82, 81], '84': [84, 83], '86': [86, 85], '89': [89, 87], '90': [90, 88], '93': [93, 91], '94': [94, 92], '96': [96, 95], '120': [120, 119], '121': [121, 119], '123': [123, 122], '124': [124, 122], '125': [125, 122], '130': [130, 129], '131': [131, 129], '132': [132, 129], '134': [134, 133], '135': [135, 133], '141': [141, 140], '142': [142, 140], '147': [147, 146], '148': [148, 146], '149': [149, 146], '151': [151, 150], '152': [152, 150], '153': [153, 154], '156': [156, 155], '163': [163, 162], '164': [164, 162]}


In [10]:
#SAc=clases c de la distribución same attendees
for index, col in df_distributions.iterrows():
    print(col["DistributionType"])


SameAttendees
SameAttendees
NotOverlap
NotOverlap
NotOverlap
NotOverlap
NotOverlap
NotOverlap
NotOverlap
NotOverlap
SameRoom
SameRoom
SameRoom
SameRoom
SameRoom
SameRoom
SameRoom
SameRoom
SameTime
SameTime
SameTime
SameTime
SameTime
SameTime
SameTime
SameTime
SameAttendees
SameAttendees
SameAttendees
SameAttendees
SameAttendees
SameAttendees
SameAttendees
SameAttendees
SameAttendees
SameAttendees
SameAttendees
SameAttendees
SameAttendees
SameAttendees
SameAttendees
SameAttendees
SameAttendees
SameAttendees
SameAttendees
SameAttendees
SameAttendees
SameAttendees
SameAttendees
SameAttendees
SameAttendees
SameAttendees
SameAttendees
SameAttendees
SameAttendees
SameAttendees
SameAttendees
SameAttendees
SameAttendees
SameAttendees
SameAttendees
SameAttendees
SameAttendees
SameAttendees
SameAttendees
SameAttendees
SameAttendees
SameAttendees
SameAttendees
SameAttendees
SameAttendees
SameAttendees
SameAttendees
SameAttendees
SameAttendees
SameAttendees
NotOverlap
NotOverlap
NotOverlap
NotOver

In [11]:
#SSc =clases c de la distribución same start
dicc_SSc = {}
for index, col in df_distributions.iterrows():
    #print(col["DistributionType"])
    if col["DistributionType"] == "SameStart":
        dicc_SSc[col["ClassesList"][0]] = [int(x) for x in col["ClassesList"]] 
print(dicc_SSc)

{}


In [12]:
#STc =clases c de la distribución same time
dicc_STc = {}
for index, col in df_distributions.iterrows():
    #print(col["DistributionType"])
    if col["DistributionType"] == "SameTime":
        dicc_STc[col["ClassesList"][0]] = [int(x) for x in col["ClassesList"]] 
print(dicc_STc)

{'81': [81, 82, 63], '83': [83, 84, 48], '85': [85, 86, 64], '87': [87, 89, 49], '88': [88, 90, 50], '91': [91, 93, 65], '92': [92, 94, 66], '95': [95, 96, 67]}


In [13]:
#SRc =clases c de la distribución same room
dicc_SRc = {}
for index, col in df_distributions.iterrows():
    if col["DistributionType"] == "SameRoom":
        dicc_SRc[col["ClassesList"][0]] = [int(x) for x in col["ClassesList"]] 
#print(dicc_SRc)

dicc_SRc2 = {}
for e in range(len(df_classes)+1):
    for i in dicc_SRc:
        #print(i,e,dicc_SRc[i])
        if str(i) == str(e):
            dicc_SRc2[e] = dicc_SRc[i]
        if e not in dicc_SRc2.keys():
            dicc_SRc2[e] = []

#dicc_SRc = dicc_SRc2
print(dicc_SRc)
##{'81': [81, 82], '83': [83, 84], '85': [85, 86], '87': [87, 88], '89': [89, 90], '91': [91, 92], '93': [93, 94], '95': [95, 96]}

{'81': [81, 82], '83': [83, 84], '85': [85, 86], '87': [87, 88], '89': [89, 90], '91': [91, 92], '93': [93, 94], '95': [95, 96]}


In [14]:
## SDc=clases c de la distribución same days
dicc_SDc = {}
for index,row in df_distributions.iterrows():
    if row["DistributionType"]=="SameDays":
        c1 = row["ClassesList"][0]
        #print(c1)
        dicc_SDc[c1] = []
        for i in range(len(row["ClassesList"])):
            dicc_SDc[c1].append(row["ClassesList"][i])
print(dicc_SDc)

{}


In [15]:
## PRc=clases c de la distribución precedence
dicc_PRc = {}
for index,row in df_distributions.iterrows():
    if row["DistributionType"]=="Precedence":
        c1 = row["ClassesList"][0]
        #print(c1)
        dicc_PRc[c1] = []
        for i in range(len(row["ClassesList"])):
            dicc_PRc[c1].append(row["ClassesList"][i])
print(dicc_PRc)

{}


In [16]:
#SPCc=id de la subparte a la que pertenece c
#dicc_subparts = {}
#dicc_SPCc = {}
#for index, row in df_subparts.iterrows():
#    dicc_subparts[row["SubpartId"]] = row["ClassIdList"]

#for subpartes in dicc_subparts:
#    for clases in dicc_subparts[subpartes]:
#        dicc_SPCc[clases] = int(subpartes)
#print(dicc_SPCc)

In [17]:
#Dc=duración de slots de clase c
dicc_Dc = {}
for index, row in df_relation_class_time.iterrows():
    dicc_Dc[row["ClassId"]] =  row["TimeLength"]
print(dicc_Dc)

{'1': 22, '2': 22, '3': 22, '4': 22, '5': 22, '6': 22, '7': 22, '8': 22, '9': 22, '10': 22, '11': 22, '12': 22, '13': 22, '14': 22, '15': 22, '16': 22, '17': 22, '18': 22, '19': 22, '20': 22, '21': 22, '22': 22, '23': 22, '24': 22, '25': 22, '26': 22, '27': 10, '28': 22, '29': 22, '30': 22, '31': 22, '32': 22, '33': 22, '34': 22, '35': 22, '36': 22, '37': 22, '38': 22, '39': 22, '40': 22, '41': 22, '42': 22, '43': 10, '44': 10, '45': 10, '46': 10, '47': 10, '48': 10, '49': 10, '50': 10, '51': 22, '52': 22, '53': 22, '54': 22, '55': 22, '56': 22, '57': 22, '58': 22, '59': 10, '60': 10, '61': 10, '62': 10, '63': 10, '64': 10, '65': 10, '66': 10, '67': 10, '68': 22, '69': 22, '70': 22, '71': 22, '72': 22, '73': 22, '74': 22, '75': 22, '76': 22, '77': 15, '78': 22, '79': 22, '80': 10, '81': 10, '82': 10, '83': 10, '84': 10, '85': 10, '86': 10, '87': 10, '88': 10, '89': 10, '90': 10, '91': 10, '92': 10, '93': 10, '94': 10, '95': 10, '96': 10, '97': 22, '98': 22, '99': 22, '100': 22, '101': 

In [18]:
#TSc=slot de inicio de la clase c
dicc_TSc = {}
for index, row in df_relation_class_time.iterrows():
    if row["ClassId"] not in dicc_TSc:
        dicc_TSc[row["ClassId"]] = []
    if row["TimeStart"] not in dicc_TSc[row["ClassId"]]:
        dicc_TSc[row["ClassId"]].append(row["TimeStart"])

for i in dicc_TSc:
    dicc_TSc[i] = [int(x) for x in dicc_TSc[i]]
print(dicc_TSc)
    

{'1': [90, 114, 138, 162, 186], '2': [90, 114, 138, 162, 186], '3': [90, 114, 138, 162, 186], '4': [90, 114, 138, 162, 186], '5': [90, 114, 138, 162, 186], '6': [90, 114, 138, 162, 186], '7': [90, 114, 138, 162, 186], '8': [90, 114, 138, 162, 186], '9': [90, 114, 138, 162, 186], '10': [90, 114, 138, 162, 186], '11': [90, 114, 138, 162, 186], '12': [90, 114, 138, 162, 186], '13': [90, 114, 138, 162, 186], '14': [90, 114, 138, 162, 186], '15': [90, 114, 138, 162, 186], '16': [90, 114, 138, 162, 186], '17': [90, 114, 138, 162, 186], '18': [90, 114, 138, 162, 186], '19': [90, 114, 138, 162, 186], '20': [90, 114, 138, 162, 186], '21': [90, 114, 138, 162, 186], '22': [90, 114, 138, 162, 186], '23': [90, 114, 138, 162, 186], '24': [90, 114, 138, 162, 186], '25': [90, 114, 138, 162, 186], '26': [90, 114, 138, 162, 186], '27': [90, 102, 114, 126, 138, 150, 162, 174, 186, 198], '28': [90, 114, 138, 162, 186], '29': [90, 114, 138, 162, 186], '30': [90, 114, 138, 162, 186], '31': [90, 114, 138, 16

In [19]:
## CPc=clase padre de la clase c
dicc_CPc = {}
for index,row in df_classes.iterrows():
    dicc_CPc[int(row["ClassId"])] = row["ClassParent"]
print(dicc_CPc)

{1: None, 2: None, 3: None, 4: None, 5: None, 6: None, 7: None, 8: None, 9: None, 10: None, 11: None, 12: None, 13: None, 14: None, 15: None, 16: None, 17: None, 18: None, 19: None, 20: None, 21: None, 22: None, 23: None, 24: None, 25: None, 26: None, 27: None, 28: None, 29: None, 30: None, 31: None, 32: None, 33: None, 34: None, 35: None, 36: None, 37: None, 38: None, 39: None, 40: None, 41: None, 42: None, 43: None, 44: None, 45: None, 46: None, 47: None, 48: None, 49: None, 50: None, 51: '43', 52: '44', 53: '45', 54: '46', 55: '47', 56: '48', 57: '49', 58: '50', 59: None, 60: None, 61: None, 62: None, 63: None, 64: None, 65: None, 66: None, 67: None, 68: '59', 69: '60', 70: '61', 71: '62', 72: '63', 73: '64', 74: '65', 75: '66', 76: '67', 77: None, 78: None, 79: None, 80: None, 81: None, 82: '81', 83: None, 84: '83', 85: None, 86: '85', 87: None, 88: None, 89: '87', 90: '88', 91: None, 92: None, 93: '91', 94: '92', 95: None, 96: '95', 97: None, 98: None, 99: None, 100: None, 101: No

In [20]:
## QSs=capacidad de estudiantes de la sala s 
dicc_QSs = {}
for index,row in df_rooms.iterrows():
    dicc_QSs[int(row["RoomId"])] = int(row["Capacity"])
print(dicc_QSs)

{1: 61, 2: 36, 3: 36, 4: 22, 5: 25, 6: 25, 7: 24, 8: 25, 9: 25, 10: 22, 11: 26, 12: 26, 13: 30}


In [21]:
## QCc=capacidad de estudiantes de la clase c
dicc_QCc = {}
for index,row in df_classes.iterrows():
    #if int(row["ClassId"]) not in dicc_QCc.keys():
        dicc_QCc[int(row["ClassId"])] = int(row["ClassLimit"])
print(dicc_QCc)

{1: 22, 2: 22, 3: 22, 4: 22, 5: 22, 6: 22, 7: 22, 8: 22, 9: 22, 10: 22, 11: 22, 12: 22, 13: 22, 14: 22, 15: 22, 16: 22, 17: 22, 18: 22, 19: 22, 20: 22, 21: 22, 22: 22, 23: 22, 24: 22, 25: 22, 26: 22, 27: 54, 28: 27, 29: 27, 30: 27, 31: 27, 32: 27, 33: 27, 34: 27, 35: 27, 36: 27, 37: 27, 38: 27, 39: 27, 40: 27, 41: 27, 42: 27, 43: 22, 44: 22, 45: 22, 46: 22, 47: 22, 48: 22, 49: 22, 50: 22, 51: 22, 52: 22, 53: 22, 54: 22, 55: 22, 56: 22, 57: 22, 58: 22, 59: 22, 60: 22, 61: 22, 62: 22, 63: 22, 64: 22, 65: 22, 66: 22, 67: 22, 68: 22, 69: 22, 70: 22, 71: 22, 72: 22, 73: 22, 74: 22, 75: 22, 76: 22, 77: 60, 78: 30, 79: 30, 80: 22, 81: 22, 82: 22, 83: 22, 84: 22, 85: 22, 86: 22, 87: 22, 88: 22, 89: 22, 90: 22, 91: 22, 92: 22, 93: 22, 94: 22, 95: 22, 96: 22, 97: 22, 98: 22, 99: 22, 100: 22, 101: 22, 102: 22, 103: 22, 104: 22, 105: 22, 106: 22, 107: 22, 108: 22, 109: 22, 110: 22, 111: 22, 112: 22, 113: 22, 114: 22, 115: 22, 116: 22, 117: 22, 118: 22, 119: 44, 120: 22, 121: 22, 122: 60, 123: 20, 

In [22]:
import math
#Agrupar capacidad de cursos

#print("Límite de las clases | Cantidad de clases con el límite")
#display(df_classes.groupby("ClassLimit").count()["ClassId"])

dicc_cursos = {}
#display(df_courses)
for index, row in df_courses.iterrows():
    dicc_cursos[row["CourseId"]] = row["ConfigIdList"]
#print(dicc_cursos)      
dicc_configs = {}
#display(df_configs)
for index, row in df_configs.iterrows():
    dicc_configs[row["ConfigId"]] = row["SubpartIdList"]
#print(dicc_configs)
dicc_subparts = {}
#display(df_subparts)
for index, row in df_subparts.iterrows():
    dicc_subparts[row["SubpartId"]] = row["ClassIdList"]
#print(dicc_subparts)
dicc_classes = {}
#display(df_classes)
for index, row in df_classes.iterrows():
    dicc_classes[row["ClassId"]] = row["ClassLimit"]
#print(dicc_classes)

dicc_lim = {}
dicc_curso_clase = {}
for p,i in dicc_cursos.items():
    for j in i:
        if j in dicc_configs.keys():
            #print(dicc_configs[j])
            for k in dicc_configs[j]:
                if k in dicc_subparts.keys():
                    #print(dicc_subparts[k])
                    for l in dicc_subparts[k]:
                        #print(l)
                        #Agregué estas lineas para hacerme un diccionario de curso con clase#
                        if l not in dicc_curso_clase:
                            dicc_curso_clase[l] = []
                        dicc_curso_clase[l].append(p)
                        ######################################################################
                        if l in dicc_classes.keys():
                            n = int(dicc_classes[l])
                            if p not in dicc_lim.keys():
                                dicc_lim[p] = [n]
                            else:
                                dicc_lim[p].append(n)
                        
#print(dicc_lim)
#print("Id de curso | Limite de alumnos")
for i in dicc_lim.keys():
    suma = 0
    contador = 0
    for j in dicc_lim[i]:
        suma += j
        contador += 1
    promedio = suma/contador
    dicc_lim[i] = int(math.ceil(promedio/10.0))*10
   #print(i, dicc_lim[i])
print(dicc_lim)

{'1': 30, '2': 40, '3': 30, '4': 30, '5': 30, '6': 40, '7': 30, '8': 30, '9': 30, '10': 30, '11': 30, '12': 30, '13': 30, '14': 30, '15': 30, '16': 30, '17': 30, '18': 20, '19': 30, '20': 30, '21': 30, '22': 40, '23': 60, '24': 30, '25': 30, '26': 30, '27': 20, '28': 20, '29': 40, '30': 40, '31': 50, '32': 40, '33': 30, '34': 30, '35': 40, '36': 30, '37': 40, '38': 20, '39': 20, '40': 30, '41': 20, '42': 20, '43': 20, '44': 40}


In [23]:
## cursos más demandados por estudiantes
dic_demanda_cursos = {}
for index,row in df_students.iterrows():
    for i in row["StudentId"]:
        for j in row["CoursesList"]:
            if j not in dic_demanda_cursos.keys():
                dic_demanda_cursos[j] = 1
                #print("agregado:",j,dic_demanda_cursos[j])
            else:
                dic_demanda_cursos[j] += 1
                #print("suma +1:",j,dic_demanda_cursos[j])
#print("Id Curso | Cantidad demandada")
for w in dic_demanda_cursos.keys():    
    print(w,dic_demanda_cursos[w])

4 499
1 1823
20 216
26 58
5 552
21 104
23 121
44 136
17 214
18 72
24 69
25 176
9 63
14 1582
16 178
3 1041
10 65
6 175
12 109
7 82
11 128
8 69
15 132
19 89
13 60
28 71
30 59
36 55
37 84
43 40
32 76
33 12
35 48
38 24


In [24]:
## Conteo de las salas mas utilizadas

dicc_conteo_clases_sala = {}
for index, row in df_relation_class_room.iterrows():
    if row["RoomId"] not in dicc_conteo_clases_sala.keys():
        dicc_conteo_clases_sala[row["RoomId"]] = 1
    else:
        dicc_conteo_clases_sala[row["RoomId"]] += 1
#print(dicc_conteo_clases_sala)
#print(len(dicc_conteo_clases_sala.keys()))
print("Sala | Cantidad de veces que se solicita")
#{k: v for k, v in sorted(dicc_conteo_clases_sala.items(), key=lambda item: item[1])}
#for i in sorted(dicc_conteo_clasess_sala)

Sala | Cantidad de veces que se solicita


In [25]:
## Dj=distribuciones de tipo j
dicc_Dj = {1:"Same Attendees",2:"Same Start",3:"Same Room",4:"Same Time",5:"Same Days",6:"Precedence"}

In [26]:
lista_I = []
for i in range(1,103):
    lista_I.append(i)
print(lista_I)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102]


In [27]:
dicc_Ij = {1:[],2:[],3:[],4:[],5:[],6:[]}
for index, row in df_distributions.iterrows():
    if row["DistributionType"] == "SameAttendees":
        dicc_Ij[1].append(row["DistributionId"])
    elif row["DistributionType"] == "SameStart":
        dicc_Ij[2].append(row["DistributionId"])
    elif row["DistributionType"] == "SameRoom":
        dicc_Ij[3].append(row["DistributionId"])
    elif row["DistributionType"] == "SameTime":
        dicc_Ij[4].append(row["DistributionId"])
    elif row["DistributionType"] == "SameDays":
        dicc_Ij[5].append(row["DistributionId"])
    elif row["DistributionType"] == "Precedence":
        dicc_Ij[6].append(row["DistributionId"])
        
print(dicc_Ij)

{1: [1, 2, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76], 2: [], 3: [11, 12, 13, 14, 15, 16, 17, 18], 4: [19, 20, 21, 22, 23, 24, 25, 26], 5: [], 6: []}


In [28]:
dicc_PTct = {}
for index, row in df_relation_class_time.iterrows():
    llave = (int(row["ClassId"]),int(index))
    dicc_PTct[llave] = int(row["TimePenalty"]) 
print(dicc_PTct)

{(1, 0): 0, (1, 1): 0, (1, 2): 0, (1, 3): 0, (1, 4): 0, (1, 5): 0, (1, 6): 0, (1, 7): 0, (1, 8): 0, (1, 9): 0, (1, 10): 0, (1, 11): 0, (1, 12): 0, (1, 13): 0, (1, 14): 0, (1, 15): 0, (1, 16): 0, (1, 17): 0, (1, 18): 0, (1, 19): 0, (2, 20): 0, (2, 21): 0, (2, 22): 0, (2, 23): 0, (2, 24): 0, (2, 25): 0, (2, 26): 0, (2, 27): 0, (2, 28): 0, (2, 29): 0, (2, 30): 0, (2, 31): 0, (2, 32): 0, (2, 33): 0, (2, 34): 0, (2, 35): 0, (2, 36): 0, (2, 37): 0, (2, 38): 0, (2, 39): 0, (3, 40): 0, (3, 41): 0, (3, 42): 0, (3, 43): 0, (3, 44): 0, (3, 45): 0, (3, 46): 0, (3, 47): 0, (3, 48): 0, (3, 49): 0, (3, 50): 0, (3, 51): 0, (3, 52): 0, (3, 53): 0, (3, 54): 0, (3, 55): 0, (3, 56): 0, (3, 57): 0, (3, 58): 0, (3, 59): 0, (4, 60): 0, (4, 61): 0, (4, 62): 0, (4, 63): 0, (4, 64): 0, (4, 65): 0, (4, 66): 0, (4, 67): 0, (4, 68): 0, (4, 69): 0, (4, 70): 0, (4, 71): 0, (4, 72): 0, (4, 73): 0, (4, 74): 0, (4, 75): 0, (4, 76): 0, (4, 77): 0, (4, 78): 0, (4, 79): 0, (5, 80): 0, (5, 81): 0, (5, 82): 0, (5, 83): 0, (

In [29]:
dicc_PScs = {}
for index, row in df_relation_class_room.iterrows():
    llave = (int(row["ClassId"]),int(row["RoomId"]))
    dicc_PScs[llave] = int(row["RoomPenalty"]) 
print(dicc_PScs)

{(1, 10): 0, (1, 11): 0, (1, 12): 0, (2, 10): 0, (2, 11): 0, (2, 12): 0, (3, 10): 0, (3, 11): 0, (3, 12): 0, (4, 10): 0, (4, 11): 0, (4, 12): 0, (5, 10): 0, (5, 11): 0, (5, 12): 0, (6, 10): 0, (6, 11): 0, (6, 12): 0, (7, 10): 0, (7, 11): 0, (7, 12): 0, (8, 10): 0, (8, 11): 0, (8, 12): 0, (9, 10): 0, (9, 11): 0, (9, 12): 0, (10, 10): 0, (10, 11): 0, (10, 12): 0, (11, 10): 0, (11, 11): 0, (11, 12): 0, (12, 10): 0, (12, 11): 0, (12, 12): 0, (13, 10): 0, (13, 11): 0, (13, 12): 0, (14, 10): 0, (14, 11): 0, (14, 12): 0, (15, 10): 0, (15, 11): 0, (15, 12): 0, (16, 10): 0, (16, 11): 0, (16, 12): 0, (17, 10): 0, (17, 11): 0, (17, 12): 0, (18, 10): 0, (18, 11): 0, (18, 12): 0, (19, 10): 0, (19, 11): 0, (19, 12): 0, (20, 10): 0, (20, 11): 0, (20, 12): 0, (21, 10): 0, (21, 11): 0, (21, 12): 0, (22, 10): 0, (22, 11): 0, (22, 12): 0, (23, 10): 0, (23, 11): 0, (23, 12): 0, (24, 10): 0, (24, 11): 0, (24, 12): 0, (25, 10): 0, (25, 11): 0, (25, 12): 0, (26, 10): 0, (26, 11): 0, (26, 12): 0, (27, 1): 0, 

In [30]:
##################### GUROBI #######################

In [31]:
from gurobipy import *
from random   import *
import numpy as np
import gurobipy as gp
from gurobipy import GRB

In [40]:
dicc_clases = {}
dicc_salas = {}
dicc_estudiantes = {}
dicc_tiempos = {}
dicc_combinaciones = {}
for e, clase in enumerate(dicc_classes):
    dicc_clases[e+1]= clase

for index, estudiante in df_students.iterrows():
    dicc_estudiantes[index+1] = estudiante["StudentId"]

for index, sala in df_rooms.iterrows():
    dicc_salas[index+1] = sala["RoomId"]

c = 0
for clase in dicc_TDc:
    for tiempo in dicc_TDc[clase]:
        if tiempo not in dicc_tiempos.values():
            dicc_tiempos[c] = tiempo
            c += 1

dicc_classes_times = {}
for index, row in df_relation_class_time.iterrows():
    if row["ClassId"] not in dicc_classes_times.keys():
        dicc_classes_times[row["ClassId"]] = []
    lista = [row["TimeDays"],row["TimeStart"],row["TimeLength"],row["TimeWeeks"],row["TimePenalty"]]
    dicc_classes_times[row["ClassId"]].append(lista)
    lista = []

dicc_configuraciones = {}
for index,row in df_configs.iterrows():
    dicc_configuraciones[int(row["ConfigId"])] = row["SubpartIdList"]
print(dicc_configuraciones)

{1: ['1'], 2: ['2', '3'], 3: ['4'], 4: ['5', '6'], 5: ['7', '8'], 6: ['9', '10'], 7: ['11'], 8: ['12', '13'], 9: ['14', '15'], 10: ['16', '17'], 11: ['18', '19'], 12: ['20', '21'], 13: ['22', '23'], 14: ['24'], 15: ['25', '26'], 16: ['27', '28'], 17: ['29'], 18: ['30', '31'], 19: ['32', '33'], 20: ['34'], 21: ['35', '36'], 22: ['37'], 23: ['38'], 24: ['39'], 25: ['40', '41'], 26: ['42', '43'], 27: ['44', '45'], 28: ['46', '47'], 29: ['48'], 30: ['49'], 31: ['50'], 32: ['51'], 33: ['52'], 34: ['53', '54'], 35: ['55'], 36: ['56'], 37: ['57'], 38: ['58'], 39: ['59'], 40: ['60'], 41: ['61'], 42: ['62'], 43: ['63'], 44: ['64']}


In [33]:
c = 0
dicc_tiempos2 = {}
for clase in dicc_TDc2:
    for tiempo in dicc_TDc2[clase]:
        if tiempo not in dicc_tiempos2.values():
            dicc_tiempos2[c] = tiempo
            c += 1
print(dicc_tiempos2)

{0: ['0100000', '90', 22, '111111111111111', '0'], 1: ['0010000', '90', 22, '111111111111111', '0'], 2: ['0001000', '90', 22, '111111111111111', '0'], 3: ['0000100', '90', 22, '111111111111111', '0'], 4: ['0100000', '114', 22, '111111111111111', '0'], 5: ['0010000', '114', 22, '111111111111111', '0'], 6: ['0001000', '114', 22, '111111111111111', '0'], 7: ['0000100', '114', 22, '111111111111111', '0'], 8: ['0100000', '138', 22, '111111111111111', '0'], 9: ['0010000', '138', 22, '111111111111111', '0'], 10: ['0001000', '138', 22, '111111111111111', '0'], 11: ['0000100', '138', 22, '111111111111111', '0'], 12: ['0100000', '162', 22, '111111111111111', '0'], 13: ['0010000', '162', 22, '111111111111111', '0'], 14: ['0001000', '162', 22, '111111111111111', '0'], 15: ['0000100', '162', 22, '111111111111111', '0'], 16: ['0100000', '186', 22, '111111111111111', '0'], 17: ['0010000', '186', 22, '111111111111111', '0'], 18: ['0001000', '186', 22, '111111111111111', '0'], 19: ['0000100', '186', 22

In [43]:
timetabling = gp.Model("Asignacion de cursos a horarios y salas")

#Conjuntos
C = dicc_clases.keys()
F = dicc_configuraciones.keys()
S = dicc_salas.keys()
T = dicc_tiempos.keys()
E = dicc_estudiantes.keys()
I = lista_I

#subconjuntos
SDisp_c = dicc_SDispc
TD_c = dicc_TDc
DF_e = dicc_DFe
UT_s = dicc_UTs
SPF_f = dicc_SPFf
SA_c = dicc_SAc 
SS_c = dicc_SSc
SR_c = dicc_SRc
PR_c = dicc_PRc
SD_c = dicc_SDc
D_j = dicc_Dj
I_j = dicc_Ij
ST_c = dicc_STc

#parámetros
CP_c = dicc_CPc
QC_c = dicc_QCc
QS_s = dicc_QSs
D_c = dicc_Dc
TS_c = dicc_TSc
PS_cs = dicc_PScs
PT_ct = dicc_PTct


x = {} 
for c in dicc_clases.keys():
    for t in dicc_tiempos.keys():
        x[c,t] = timetabling.addVar(vtype=gp.GRB.BINARY, name='x_'+str(c)+','+str(t))

y = {}
for c in dicc_clases.keys():
    for s in dicc_salas.keys():
        y[c,s] = timetabling.addVar(vtype=gp.GRB.BINARY, name='y_'+str(c)+','+str(s))

z = {}
for e in dicc_estudiantes.keys():
    for c in dicc_clases.keys():
        z[e,c] = timetabling.addVar(vtype=gp.GRB.BINARY, name='z_'+str(e)+','+str(c))

w = {}
for e in dicc_estudiantes.keys():
    for f in dicc_configuraciones.keys():
        w[e,f] = timetabling.addVar(vtype=gp.GRB.BINARY, name='w_'+str(e)+','+str(f))
        
xs = {}
for c in dicc_clases.keys():
    for t in dicc_tiempos.keys():
        xs[c,t] = timetabling.addVar(vtype=gp.GRB.BINARY, name='xs_'+str(c)+','+str(t))
        
aux = {}
for j in D_j.keys():
    for i in I:
        aux[j,i] = timetabling.addVar(vtype=gp.GRB.BINARY, name="aux_"+str(j)+","+str(i))

#Función Objetivo
timetabling.setObjective((gp.quicksum(x[c,t]*PT_ct[(c,t)] for c in C for t in TD_c[c])+gp.quicksum(y[c,s]*PS_cs[(c,s)] for c in C for s in SDisp_c[c])),GRB.MINIMIZE)

#Restricciones
#A cada clase se le debe ser asignado un tiempo
timetabling.addConstrs(((gp.quicksum(x[c,t] for t in TD_c.keys()) == 1) for c in C), name = "R1")
#Cada clase debe ser asignada a una sala (donde sea aplicable)
timetabling.addConstrs(((gp.quicksum(y[c,s] for s in SDisp_c[c]) == 1) for c in C), name = "R2")
#Cada estudiante debe ser asignado a una configuración de curso para cada curso que demanda
timetabling.addConstrs(((gp.quicksum(w[e,f] for f in DF_e[e]) == 1) for e in E), name = "R3")
#La capacidad de cada clase en términos del número de estudiantes debe ser satisfecha
timetabling.addConstrs(((gp.quicksum(z[e,c] for e in E) <= QC_c[c]*x[c,t]) for c in C for t in TD_c[c]), name = "R4")
#Una sala no puede ser usada cuando no está disponible
#timetabling.addConstrs(((y[c,s] + x[c,t] <= 1) for c in C for s in SDisp_c[c] for t in UT_s[s]), name = "R5")
#Dos clases no pueden ser asignadas en un mismo tiempo en la misma sala
timetabling.addConstrs(((x[c1,t1] + y[c1,s1] + x[c1,t2] + y[c2,s2] <= 3) for c1 in C for c2 in C for t1 in TD_c[c1] for t2 in TD_c[c2] for s1 in SDisp_c[c1] for s2 in SDisp_c[c2] if c1 != c2 if t1 == t2 if s1 == s2), name = "R6")
#SAME ATTENDEES
timetabling.addConstrs(((x[c1,t1] + x[c2,t2] == 2) for c1 in C for c2 in C for t1 in TD_c[c1] for t2 in TD_c[c2] if c1!=c2 if t1==t2 if c1 in SA_c[c2] or c2 in SA_c[c1]), name = "R7")
#SAME START
#timetabling.addConstrs(((xs[c1,t1] + xs[c2,t2] == 2) for c1 in C for c2 in C for t1 in TD_c[c1] for t2 in TD_c[c2] if c1 != c2 if dicc_tiempos2[t1][1]==dicc_tiempos2[t2][1] if c1 in SS_c[c2] or c2 in SS_c[c1]), name = "R8")
#SAME ROOMS
#timetabling.addConstrs(((y[c1,s1] + y[c2,s2] == 2) for c1 in C for c2 in C for s1 in SDisp_c[c1] for s2 in SDisp_c[c2] if c1!=c2 if s1==s2 if c1 in SR_c[c2] or c2 in SR_c[c1]), name = "R9")
#SAME TIME
timetabling.addConstrs(((x[c1,t1] + x[c2,t2] == 2) for c1 in C for c2 in C for t1 in TD_c[c1] for t2 in TD_c[c2] if c1!=c2 if t1==t2 if c1 in ST_c[c2] or c2 in ST_c[c1]), name = "R10")
#SAME DAYS
#timetabling.addConstrs(((xs[c1,t1] <= gp.quicksum(xs[c2,t2] for t2 in TD_c[c2])) for c1 in C for c2 in C for t1 in TD_c[c1] if c1!=c2 if dicc_tiempos2[t1][0]==dicc_tiempos2[t2][0] if c1 in SD_c[c2] or c2 in SD_c[c1]), name = "R11")
#timetabling.addConstrs(((xs[c1,t1] == xs[c2,t2+D_c[c2]+1]) for c1 in PR_c[c1] for c2 in PR_c[c2] for t1 in TD_c[c1] for t2 in TD_c[c2] if c1!=c2), name = "R13")


#Restricciones blandas
timetabling.addConstrs((((x[c1,t1] - x[c2,t2])*(x[c1,t1] - x[c2,t2]) == Aux[1,i]*Aux[1,i]) for c1 in C for c2 in C for t1 in TD_c[c1] for t2 in TD_c[c2] if c1 != c2 if t1 == t2 if c1 in SA_c[c2] or c2 in SA_c[c1]), name = "SameAttendees B")

#timetabling.addConstrs((xs[c1,t1]+xs[c2,t2]-2 == Aux[2,i] for c1 in SS_c[c1] for c2 in SS_c[c2] for t1 in TD_c[c1] for t2 in TD_c[c2] if c1 != c2 if dicc_tiempos2[t1][1]==dicc_tiempos2[t2][1]), name = "Same Start B")

#timetabling.addConstrs((y[c1,s1]+y[c2,s2]-2 == Aux[3,i] for c1 in SR_c[c1] for c2 in SR_c[c2] for s1 in SDisp_c[c1] for s2 in SDisp_c[c2] if c1 != c2 if s1 == s2), name = "Same Rooms B")

#timetabling.addConstrs((x[c1,t1]+x[c2,t2]-2 == Aux4 for c1 in SS_c[c1] for c2 in SS_c[c2] for t1 in TD_c[c1] for t2 in TD_c[c2] if c1!=c2 if t1==t2), name = "Same Time B")

#timetabling.addConstrs((xs[c1,t1]-gp.quicksum(xs[c2,t2] for t2 in TD_c[c2]) <= Aux5 for c1 in SD_c[c1] for c2 in SD_c[c2] for t1 in TD_c[c1] if c1!=c2 if dicc_tiempos2[t1][0]==dicc_tiempos2[t2][0]), name = "Same Days B")

#timetabling.addConstrs((xs[c1,t1]-xs[c2,t1+dc+1] == Aux8[8,i] for c1 in PR_c[c1] for c2 in PR_c[c2] for t1 in TD_c[c1] if c1!=c2), name = "Precedence B")

## Relación entre variables

timetabling.addConstrs((x[c,t1] <= xs[c,t2] for c in C for t1 in TD_c[c] for t2 in TD_c[c] if t1 == t2), name ="RelacionX-XSn1")

timetabling.addConstrs(((xs[c,t] <= gp.quicksum(x[c,t] for t in TD_c[c])) for c in C), name = "RelacionX-XSn2")

#timetabling.addConstrs(((x[c,t] == y[c,s]) for c in C for t in TD_c[c] for s in SDisp_c[c]), name = "RelacionXS-Y")
timetabling.addConstrs(((z[e,c] == w[e,f]) for e in E for c in C for f in F), name = "RelacionZ-W")


timetabling.optimize()
#for v in timetabling.getVars():
#    print('%s %g' % (v.varName, v.x))

GurobiError: Out of memory

In [51]:
timetabling.write("file_infeasible.lp")
timetabling.write("file_infeasible.mps")